In [5]:
#НОРМЫ ПО ПОНЕДЕЛЬНИКАМ

import pandas as pd

# Загружаем нужные столбцы
cols = ['Номер из NAV','Адрес магазина','дата планового открытия','статус','ИФ кольца','ИФ КОЛЬЦА ОБРУЧ',
        'ИФ печатки','ИФ серьги','ИФ подвес культ','ИФ подвес декор','БК кольца','БК кольца обруч','БК печатки',
        'БК серьги','БК подвес культ','БК подвес декор','ИТОГО, нормы в штуках, без накоплений']

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols)

# Убираем ненужную инфу
df.rename({'ИТОГО, нормы в штуках, без накоплений':"sum_all"}, axis=1, inplace=True)
df = df[df['sum_all'] > 0]
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружен'])]

# Добавляем сумму по ИФ\БК и оставляем этот столбец вместо столбцов с нормами по иф\бк
sum_cols = [x for x in cols if 'ИФ' in x or 'БК' in x ]
df['sum_gold'] = df[sum_cols].sum(axis=1)
df.drop(axis=1, columns=sum_cols, inplace=True)

# Количество открытых и к открытию всего и только по золоту
shops_count = pd.DataFrame({
    'Магов открыто': [df.sum_all[df.статус !='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус !='склад')].count()],
    'Магов к открытию': [df.sum_all[df.статус =='склад'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус =='склад')].count()],
    'Итого магов': [df.sum_all.count(),df.sum_gold[df.sum_gold > 0].count()]})

# Список новых магов с датами
new_shops = df[['Номер из NAV', 'Адрес магазина', 'дата планового открытия']][(df.sum_gold > 0) & (df.статус =='склад')]
new_shops.sort_values('дата планового открытия', inplace=True)

# Нормы ИФ/БК по ТПГ
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

norm_all = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)

lst = norm_all.columns.tolist()

cols = (
    [lst[lst.index('Название из NAV')]] +
    lst[lst.index(1001):lst.index(1102)+1] +
    lst[lst.index(1501):lst.index(1597)+1]
)
norm_all = norm_all[cols]
gold_shops = df[['Номер из NAV', 'статус']][df.sum_gold > 0]
merged = pd.merge(gold_shops, norm_all, left_on='Номер из NAV', right_on='Название из NAV', how = 'left')

del merged['Название из NAV'], merged['Номер из NAV']

# Все нецифры меняю на ноль. из ексель подтягивается значок '-'
nostatus = [i for i in merged.columns.to_list() if i !='статус']
merged[nostatus] = merged[nostatus].apply(pd.to_numeric, errors='coerce').fillna(0)


merged = merged.groupby(by='статус').sum()
merged = merged.T
merged['sum_all'] = merged.sum(axis=1)
merged = merged[['открыт', 'sum_all']]

# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    shops_count.to_excel(writer, sheet_name='количество', index=False)
    new_shops.to_excel(writer, sheet_name='новые_маги', index=False)
    merged.to_excel(writer, sheet_name='новые_нормы')

In [1]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ И ЗОДИАКАМ 
import pandas as pd

# Читаем столбцы с нормами по буквам и зодиакам из реестра
cols = [2, *range(368,374), *range(469,475)]
link = r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm'    
norm = pd.read_excel(link, sheet_name='НОРМЫ 17.12', skiprows=4, usecols=cols)

# Обрезаем пустые строки, делим на буквы и зодиаки
last_row = norm.index[norm['Название из NAV'].isna()][0]-1
norm = norm.loc[:last_row]
norm.set_index('Название из NAV',inplace=True)

b_columns = [1088, 1089, 1090, 1586, 1587, 1588]
z_columns = [1091, 1092, 1093, 1589, 1590, 1591]
norm_b = norm[b_columns]
norm_z = norm[z_columns]

norm_b = norm_b.apply(pd.to_numeric, errors='coerce').fillna(0)
norm_z = norm_z.apply(pd.to_numeric, errors='coerce').fillna(0)

norm_b['sum_if'], norm_b['sum_bk'] = norm_b[[1088, 1089, 1090]].sum(axis=1), norm_b[[1586, 1587, 1588]].sum(axis=1)
norm_z['sum_if'], norm_z['sum_bk'] = norm_z[[1091, 1092, 1093]].sum(axis=1), norm_z[[1589, 1590, 1591]].sum(axis=1)

with pd.ExcelWriter (r'C:\Остатки\norm_bz.xlsx') as writer:
    norm_b.to_excel(writer, sheet_name='norm_b')
    norm_z.to_excel(writer, sheet_name='norm_z')

In [1]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ И ДАТЫ ОТКРЫТИЯ

import pandas as pd
import numpy as np
pd.set_option('use_inf_as_na', True)

#выгружаем датафрэйм с актуальными нормами ИФ БК из реестра магов 
cols_df = ['Номер из NAV', 'Адрес магазина', 'дата планового открытия','статус', 'ИФ кольца', 'ИФ КОЛЬЦА ОБРУЧ', 'ИФ печатки', 'ИФ серьги', 
        'ИФ подвес культ', 'ИФ подвес декор', 'БК кольца', 'БК кольца обруч', 'БК печатки', 'БК серьги', 
        'БК подвес культ', 'БК подвес декор']
      
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols_df)

#выбираем открытые маги с суммой норм по иф/бк больше нуля
df = df[df.статус.str.contains('откр|отгр|склад', na=False, case=False)]
df['сумма'] = df.iloc[:, 3:].sum(axis=1)
df = df[df['сумма'] > 0]


all_shops = df.drop(columns=['статус','сумма','дата планового открытия'])
all_shops = all_shops.rename(columns={'Номер из NAV':'Код'})
all_shops['Код'] = all_shops['Код'].astype(np.int64)

# делим на два отдельных иф и бк
cols = all_shops.columns.tolist()
shops_if = all_shops[[*cols[:5],cols[7], cols[6], cols[5]]]
shops_bk = all_shops[[*cols[:2], *cols[8:11], cols[13],cols[12],cols[11]]]

#выгружаем из файла "наполняемость" актуальную структуру магов иф/бк, добавляет туда нормы и новые маги из датафрэйма выше
shops_if_list = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx',
    sheet_name='ИФ', skiprows=3, usecols=[2],keep_default_na=False)

shops_bk_list = pd.read_excel(
    r'C:\Остатки\наполняемость с 10012020.xlsx',
    sheet_name='БК', skiprows=3, usecols=[2],keep_default_na=False)

# список новых магов которые надо добавить
add_to_if = pd.DataFrame({'Код':[x for x in shops_if['Код'].tolist() if x not in shops_if_list['Код'].tolist()]})
add_to_bk = pd.DataFrame({'Код':[x for x in shops_bk['Код'].tolist() if x not in shops_bk_list['Код'].tolist()]})

# добавляем новые маги в структуру
shops_if_list = shops_if_list.append(add_to_if, sort=False)
shops_bk_list = shops_bk_list.append(add_to_bk, sort=False)
all_shops_lst = list(set(shops_if_list.Код.tolist()).union(set(shops_bk_list.Код.tolist())))
all_shops_lst.append(7001)

#соединяем "структуру" и нормы добавляем итоги
shops_if = pd.merge(shops_if_list, shops_if, how='left', on='Код')
shops_bk = pd.merge(shops_bk_list, shops_bk, how='left', on='Код')

shops_if['Итого'] = shops_if.iloc[:, 1:].sum(axis=1)
shops_bk['Итого'] = shops_bk.iloc[:, 1:].sum(axis=1)

# загружаем и соединяем заборные листы и остатки
cols = ['КодСклада', 'СерийныйНомер', 'Тип1', 'ТоварноеНаправление']
stock = pd.read_csv(r'C:\анаконда\tmp.csv', delimiter=';', encoding='utf-8', usecols=cols)
stock = stock[stock.КодСклада.isin(all_shops_lst)]

stock_7001 = stock[stock.КодСклада == 7001]
stock = stock[stock.КодСклада != 7001]

zab = pd.read_excel(r'C:\анаконда\zab.xlsx', skiprows=4)
zab = zab[["Серийный Но.","Склад-приемник","Номер ЗЛ"]].sort_values(by="Номер ЗЛ",ascending = False)
zab.columns = ['СерийныйНомер','КодСклада','ЗЛ']
del zab['ЗЛ']
 
stock_7001 = stock_7001.drop(columns='КодСклада').merge(zab,on=['СерийныйНомер'],how = 'inner')

stock_full = pd.concat([stock_7001,stock])
stock_full.rename(columns={'КодСклада': 'Код'}, inplace=True)

# формируем одинаковые датасеты нормы-остатки, считаем наполняемость
k = ['КОЛЬЦО', 'СЕРЬГИ', 'ПОДВЕС ДЕКОРАТИВНЫЙ', 'ПОДВЕС КУЛЬТОВЫЙ', 
     'КОЛЬЦО ПЕЧАТКА', 'КОЛЬЦО ОБРУЧАЛЬНОЕ', 'ПИРСИНГ', 'СЕРЬГИ-КОНГО']

v = ['кольца', 'серьги', 'подвес декор', 'подвес культ',
     'печатки', 'КОЛЬЦА ОБРУЧ', 'серьги', 'серьги']

if_dict = {key: 'ИФ ' + value for key, value in zip(k,v)}#словарь для переименования тип1 в тг
bk_dict = {key: 'БК ' + value.lower() if value == 'КОЛЬЦА ОБРУЧ' else 'БК ' + value for key, value in zip(k,v)}

stock_full = stock_full[stock_full['Тип1'].isin(k)]

stock_if = stock_full[stock_full.ТоварноеНаправление == 'ИФ']
stock_if['Тип1'] = stock_if.Тип1.apply(lambda x: if_dict[x])
stock_if = stock_if.drop(columns='ТоварноеНаправление').pivot_table(index='Код', columns='Тип1', aggfunc='count')
stock_if.columns = stock_if.columns.droplevel(0)
stock_if.reset_index(inplace=True)
stock_if = shops_if[['Код']].merge(stock_if, on='Код', how='left')
stock_if.fillna(0, inplace=True)
stock_if['Итого'] = stock_if.iloc[:, 1:].sum(axis=1)

stock_bk = stock_full[stock_full.ТоварноеНаправление == 'БК']
stock_bk['Тип1'] = stock_bk.Тип1.apply(lambda x: bk_dict[x])
stock_bk = stock_bk.drop(columns='ТоварноеНаправление').pivot_table(index='Код', columns='Тип1', aggfunc='count')
stock_bk.columns = stock_bk.columns.droplevel(0)
stock_bk.reset_index(inplace=True)
stock_bk = shops_bk[['Код']].merge(stock_bk, on='Код', how='left')
stock_bk.fillna(0, inplace=True)
stock_bk['Итого'] = stock_bk.iloc[:, 1:].sum(axis=1)

if_cols = [x for x in shops_if.columns.tolist() if x != 'Адрес магазина']
bk_cols = [x for x in shops_bk.columns.tolist() if x != 'Адрес магазина']
stock_if = stock_if[if_cols]
stock_bk = stock_bk[bk_cols]

if_fillrate = stock_if.iloc[:, 1:] / shops_if.iloc[:, 2:]
#if_fillrate['Код'] = stock_if.iloc[:, 0]

bk_fillrate = stock_bk.iloc[:, 1:] / shops_bk.iloc[:, 2:]
#bk_fillrate['Код'] = stock_bk.iloc[:, 0]

shops_if = pd.concat([shops_if, stock_if.iloc[:, 1:], if_fillrate], axis=1)#для проверки
shops_bk = pd.concat([shops_bk, stock_bk.iloc[:, 1:], bk_fillrate], axis=1)#для проверки

#удаляем индекс и сохраняем на отдельные вкладки
shops_if.set_index('Код', drop=True,inplace=True)
shops_bk.set_index('Код', drop=True,inplace=True)

#создаем датафрейм из магов с датами открытия последние 90 дней
from datetime import date, timedelta

three_month_ago = date.today() - timedelta(90)
df['дата планового открытия'] = df['дата планового открытия'].apply(lambda x:x.date())#чтобы ошибка не выпадала о формате
dates = df[['Номер из NAV', 'дата планового открытия']][df['дата планового открытия'] > three_month_ago]
dates.set_index('Номер из NAV', drop=True,inplace=True)

#сохраняем все в эксель
with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
) as writer:
    shops_if.to_excel(writer, sheet_name='ИФ')
    shops_bk.to_excel(writer, sheet_name='БК')
    dates.to_excel(writer, sheet_name='даты')

C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Dotsenko.Semen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [1]:
# СПИСОК НОРМ ПО ТГ

import pandas as pd
import numpy as np

#читает файл с реестром
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn = df.iloc[0:1,28:74]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

In [1]:
#СЧИТАЕТ КОЛИЧЕСТВО МАГАЗОВ 3 МЕСЯЦА ДО СЕГОДНЯ И ПОСЛЕ

import pandas as pd
import numpy as np

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=([2, 3, 6, 18] + list(range(28, 41))), parse_dates=[6] )

# выбираем маги с нужным статусом, с нормами по тн больше нуля 
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружено'])]
df[['дата планового открытия']] = df['дата планового открытия'].fillna(df['дата планового открытия'].min())
df['sum'] = df.iloc[ : , 5:].sum(axis=1)
df = df[df['sum']>0]


In [3]:
df[['дата планового открытия']][df['дата планового открытия'] > '2020-07-01'].sort_values(['дата планового открытия'])


,дата планового открытия
947,2020-07-12
951,2020-08-01
950,2020-08-10
954,2020-08-10
952,2020-08-11
953,2020-08-25
968,2020-09-30
971,2020-10-01
967,2020-10-01
966,2020-10-01
